In [4]:
from ultralytics import YOLO
import cv2
from pathlib import Path

def run_detection_on_video(model_path, video_path, output_path=None, conf_thres=0.25):
    """
    Run detection on a video using YOLOv8 model.
    
    Args:
        model_path (str): Path to the YOLOv8 model
        video_path (str): Path to input video
        output_path (str, optional): Path to save output video
        conf_thres (float): Confidence threshold for detections
    """
    try:
        # Load the YOLOv8 model
        model = YOLO(model_path)
        
        # Open video capture
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error: Could not open video.")
            return
            
        # Get video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Setup output video writer
        if output_path is None:
            output_path = str(Path(video_path).with_stem(Path(video_path).stem + '_detected'))
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        
        try:
            # Process the video frame by frame
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Run YOLOv8 inference on the frame
                results = model(frame, conf=conf_thres)[0]
                
                # Visualize the results on the frame
                annotated_frame = results.plot()
                
                # Write the frame to output video
                out.write(annotated_frame)
                
                # Display the frame
                cv2.imshow("YOLOv8 Detection", annotated_frame)
                
                # Break the loop if 'q' is pressed
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            # Release resources
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    # Example usage
    MODEL_PATH = "C:/Users/DELL/Downloads/best.pt"  # Path to your YOLOv8x model
    VIDEO_PATH = "C:/Users/DELL/Desktop/accident detection/numberplate/tripletest.mp4"  # Path to your input video
    OUTPUT_PATH = "C:/Users/DELL/Desktop/accident detection/numberplate/bestptoutput.mp4"
    
    run_detection_on_video(
        model_path=MODEL_PATH,
        video_path=VIDEO_PATH,
        output_path=OUTPUT_PATH,
        conf_thres=0.25  # Adjust confidence threshold as needed
    )


0: 384x640 2 Hardhats, 2 NO-Hardhats, 2 NO-Masks, 6 NO-Safety Vests, 9 Persons, 1 machinery, 30.0ms
Speed: 0.0ms preprocess, 30.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hardhat, 3 NO-Hardhats, 2 NO-Masks, 6 NO-Safety Vests, 8 Persons, 1 machinery, 8.5ms
Speed: 0.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hardhat, 3 NO-Hardhats, 2 NO-Masks, 5 NO-Safety Vests, 6 Persons, 1 machinery, 1 vehicle, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hardhat, 5 NO-Hardhats, 2 NO-Masks, 6 NO-Safety Vests, 7 Persons, 9.2ms
Speed: 0.0ms preprocess, 9.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Hardhat, 3 NO-Hardhats, 3 NO-Masks, 5 NO-Safety Vests, 8 Persons, 1 machinery, 4.6ms
Speed: 3.2ms preprocess, 4.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Hardhats, 

In [2]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import numpy as np

def run_detection_on_video(model_path, video_path, output_path=None, conf_thres=0.25, selected_classes=['With Helmet', 'Without Helmet']):
    """
    Run detection on a video using YOLOv8 model with class filtering.
     'Hardhat', 'NO-Hardhats'
    Args:
        model_path (str): Path to the YOLOv8 model
        video_path (str): Path to input video
        output_path (str, optional): Path to save output video
        conf_thres (float): Confidence threshold for detections
        selected_classes (list): List of class names to detect
    """
    try:
        # Load the YOLOv8 model
        model = YOLO(model_path)
        
        # Get class indices for selected classes
        class_indices = [model.names.get(i) for i in range(len(model.names))]
        selected_class_indices = [i for i, name in enumerate(class_indices) if name in selected_classes]
        
        # Open video capture
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error: Could not open video.")
            return
            
        # Get video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Setup output video writer
        if output_path is None:
            output_path = str(Path(video_path).with_stem(Path(video_path).stem + '_detected'))
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        
        try:
            # Process the video frame by frame
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Run YOLOv8 inference on the frame
                results = model(frame, conf=conf_thres)[0]
                
                # Filter detections for selected classes
                boxes = results.boxes
                filtered_boxes = boxes[np.isin(boxes.cls.cpu(), selected_class_indices)]
                results.boxes = filtered_boxes
                
                # Visualize the filtered results on the frame
                annotated_frame = results.plot()
                
                # Write the frame to output video
                out.write(annotated_frame)
                
                # Display the frame
                cv2.imshow("YOLOv8 Detection", annotated_frame)
                
                # Break the loop if 'q' is pressed
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            # Release resources
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    # Example usage
    MODEL_PATH = "C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt"  # Path to your YOLOv8x model
    VIDEO_PATH = "C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4"  # Path to your input video
    OUTPUT_PATH = "C:/Users/DELL/Desktop/only traffic project/helmet detection/outputBEST1.mp4"
    
    
    # Specify which classes you want to detect
    SELECTED_CLASSES = ['With Helmet', 'Without Helmet']
    
    run_detection_on_video(
        model_path=MODEL_PATH,
        video_path=VIDEO_PATH,
        output_path=OUTPUT_PATH,
        conf_thres=0.25,  # Adjust confidence threshold as needed
        selected_classes=SELECTED_CLASSES
    )

    #['With Helmet', 'Without Helmet']


0: 384x640 8 With_Helmets, 16.5ms
Speed: 0.0ms preprocess, 16.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 With_Helmets, 4.0ms
Speed: 17.5ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 17.0ms
Speed: 0.0ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 16.7ms
Speed: 1.0ms preprocess, 16.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 16.6ms
Speed: 1.3ms preprocess, 16.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 16.5ms
Speed: 0.0ms preprocess, 16.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 With_Helmets, 16.8ms
Speed: 1.6ms preprocess, 16.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 With_Helmets, 1 Without_Helmet, 17.2ms
Speed: 0.0ms preproce

# all tests 

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [3]:
import cv2
import math
import cvzone
from ultralytics import YOLO

# Initialize video capture
video_path = "C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4"
cap = cv2.VideoCapture(video_path)

# Load YOLO model with custom weights
model = YOLO("C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt")

# Define class names
classNames = ['With_Helmet', 'Without_Helmet']

# For the use of Webcam
# Open the webcam (use 0 for the default camera, or 1, 2, etc. for additional cameras)
# cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


0: 384x640 8 With_Helmets, 27.4ms
Speed: 1.0ms preprocess, 27.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 With_Helmets, 8.4ms
Speed: 2.0ms preprocess, 8.4ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 17.0ms
Speed: 2.6ms preprocess, 17.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 18.0ms
Speed: 1.9ms preprocess, 18.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 17.7ms
Speed: 2.1ms preprocess, 17.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 16.8ms
Speed: 2.1ms preprocess, 16.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 With_Helmets, 17.9ms
Speed: 0.0ms preprocess, 17.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 With_Helmets, 1 Without_Helmet, 18.3ms
Speed: 1.6ms preproce

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


: 

In [2]:
import cv2
import math
import cvzone
from ultralytics import YOLO

# Initialize video capture
video_path = "C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer to save output
output_path = "C:/Users/DELL/Desktop/only traffic project/helmet detection/testOUTPUT.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define codec
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Load YOLO model with custom weights
model = YOLO("C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt")

# Define class names
classNames = ['With_Helmet', 'Without_Helmet']

# Process video frames
while True:
    success, img = cap.read()
    if not success:
        break  # End of video

    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            # Add bounding box and label
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)

    # Write the frame to the output video
    out.write(img)

    # Display the frame (optional)
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved as {output_path}")




0: 384x640 8 With_Helmets, 40.6ms
Speed: 0.0ms preprocess, 40.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 With_Helmets, 10.3ms
Speed: 3.2ms preprocess, 10.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 20.7ms
Speed: 0.0ms preprocess, 20.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 17.6ms
Speed: 1.2ms preprocess, 17.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 17.7ms
Speed: 1.6ms preprocess, 17.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 18.0ms
Speed: 3.0ms preprocess, 18.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 With_Helmets, 16.8ms
Speed: 0.0ms preprocess, 16.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 With_Helmets, 1 Without_Helmet, 18.8ms
Speed: 0.0ms preproc

In [3]:
# people + motorcyle + helmet
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt')  # Model with helmet detection capability

# Set the video source (0 for webcam, or provide a video file path)
video_source = "C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4"  # Update to video path if needed
cap = cv2.VideoCapture(video_source)

# Define the output video writer (optional)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter("C:/Users/DELL/Desktop/only traffic project/helmet detection/tesyOUTPUT1.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

# Define colors for classes
colors = {
    'person': (0, 255, 0),  # Green for person
    'motorcycle': (255, 0, 0),  # Blue for motorcycle
    'helmet': (0, 0, 255),  # Red for helmet
}

# Class IDs for COCO dataset
person_class_id = 0  # Replace with your model's person class ID
motorcycle_class_id = 3  # Replace with your model's motorcycle class ID
helmet_class_id = 1  # Replace with your model's helmet class ID

# Function to calculate IoU (Intersection over Union)
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union = area1 + area2 - intersection
    return intersection / union if union > 0 else 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    person_count = 0
    motorcycle_count = 0
    helmet_count = 0
    helmetless_count = 0

    person_boxes = []
    helmet_boxes = []

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
            conf = float(box[4])                # Confidence score (as float)
            cls = int(box[5])                   # Class ID (as integer)

            # Detect person
            if cls == person_class_id:
                person_count += 1
                person_boxes.append((x1, y1, x2, y2))
                label = f'Person {conf:.2f}'
                color = colors['person']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Detect motorcycle
            elif cls == motorcycle_class_id:
                motorcycle_count += 1
                label = f'Motorcycle {conf:.2f}'
                color = colors['motorcycle']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Detect helmet
            elif cls == helmet_class_id:
                helmet_count += 1
                helmet_boxes.append((x1, y1, x2, y2))
                label = f'Helmet {conf:.2f}'
                color = colors['helmet']
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Check if persons are wearing helmets
    for person_box in person_boxes:
        helmet_found = False
        for helmet_box in helmet_boxes:
            iou = calculate_iou(person_box, helmet_box)
            if iou > 0.3:  # Adjust IoU threshold as needed
                helmet_found = True
                break
        if not helmet_found:
            helmetless_count += 1
            x1, y1, x2, y2 = person_box
            cv2.putText(frame, 'No Helmet', (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Display the counts on the frame
    cv2.putText(frame, f'Persons Detected: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Motorcycles Detected: {motorcycle_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f'Helmetless Persons: {helmetless_count}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the video frame
    cv2.imshow('Helmet Detection', frame)

    # Write the frame to output video (optional)
    out.write(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 8 With_Helmets, 34.0ms
Speed: 0.0ms preprocess, 34.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 With_Helmets, 16.0ms
Speed: 1.9ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 17.2ms
Speed: 1.2ms preprocess, 17.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 16.2ms
Speed: 1.1ms preprocess, 16.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 13.5ms
Speed: 1.1ms preprocess, 13.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 17.1ms
Speed: 0.9ms preprocess, 17.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 With_Helmets, 18.4ms
Speed: 0.0ms preprocess, 18.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 With_Helmets, 1 Without_Helmet, 16.2ms
Speed: 0.5ms preproc

In [6]:
from ultralytics import YOLO

# Load your model
model = YOLO('C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt')

# Get the names dictionary
names = model.names

# Print all class names with their indices
for idx, name in names.items():
    print(f"Class {idx}: {name}")

Class 0: With_Helmet
Class 1: Without_Helmet


In [1]:
#with basic confidence 
import cv2
import numpy as np
from ultralytics import YOLO

class HelmetDetector:
    def __init__(self, model_path, video_path, output_path):
        # Initialize YOLO model
        self.model = YOLO(model_path)
        
        # Initialize video capture
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError("Error opening video file")
            
        # Get video properties
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer
        self.out = cv2.VideoWriter(output_path, 
                                 cv2.VideoWriter_fourcc(*'mp4v'), 
                                 self.fps, 
                                 (self.frame_width, self.frame_height))
        
        # Colors for visualization
        self.colors = {
            'With_Helmet': (0, 255, 0),    # Green
            'Without_Helmet': (0, 0, 255)   # Red
        }
        
        # Counters for statistics
        self.stats = {
            'With_Helmet': 0,
            'Without_Helmet': 0
        }

    def draw_detection_box(self, frame, box, cls, conf):
        """Draw detection box with label and confidence"""
        x1, y1, x2, y2 = map(int, box)
        color = self.colors[cls]
        
        # Draw box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Create label with class and confidence
        label = f'{cls} {conf:.2f}'
        
        # Get label size
        (label_width, label_height), _ = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        
        # Draw label background
        cv2.rectangle(frame, 
                     (x1, y1 - label_height - 10), 
                     (x1 + label_width + 10, y1), 
                     color, 
                     -1)
        
        # Draw label text
        cv2.putText(frame, label, 
                    (x1 + 5, y1 - 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                    (255, 255, 255), 
                    2)

    def draw_stats(self, frame):
        """Draw detection statistics on frame"""
        # Background for stats
        cv2.rectangle(frame, 
                     (10, 10), 
                     (250, 100), 
                     (0, 0, 0), 
                     -1)
        
        # Draw stats text
        y_pos = 40
        for cls, count in self.stats.items():
            color = self.colors[cls]
            text = f'{cls}: {count}'
            cv2.putText(frame, text, 
                       (20, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       0.8, color, 2)
            y_pos += 30

    def process_video(self):
        """Process the video and detect helmets"""
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
                
            # Reset counters for each frame
            self.stats = {
                'With_Helmet': 0,
                'Without_Helmet': 0
            }
            
            # Run detection with increased confidence threshold
            results = self.model(frame, conf=0.5)
            
            # Process detections
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0]
                    
                    # Get class and confidence
                    cls = self.model.names[int(box.cls[0])]
                    conf = float(box.conf[0])
                    
                    # Update statistics
                    self.stats[cls] += 1
                    
                    # Draw detection box
                    self.draw_detection_box(frame, [x1, y1, x2, y2], cls, conf)
            
            # Draw statistics
            self.draw_stats(frame)
            
            # Show frame
            cv2.imshow('Helmet Detection', frame)
            
            # Write frame to output video
            self.out.write(frame)
            
            # Break loop on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release resources
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()

def main():
    # Paths configuration
    model_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt'
    video_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4'
    output_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/testOUTPUT2.mp4'
    
    # Create and run detector
    detector = HelmetDetector(model_path, video_path, output_path)
    detector.process_video()

if __name__ == "__main__":
    main()


0: 384x640 7 With_Helmets, 45.0ms
Speed: 0.0ms preprocess, 45.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 With_Helmets, 16.2ms
Speed: 2.1ms preprocess, 16.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 With_Helmets, 17.6ms
Speed: 3.1ms preprocess, 17.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 With_Helmets, 17.8ms
Speed: 2.0ms preprocess, 17.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 With_Helmets, 17.8ms
Speed: 1.0ms preprocess, 17.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 With_Helmets, 18.1ms
Speed: 1.1ms preprocess, 18.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 With_Helmets, 17.4ms
Speed: 2.6ms preprocess, 17.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 With_Helmets, 17.3ms
Speed: 2.0ms preprocess, 17.3ms inference

In [8]:
#with 70% confidence 
import cv2
import numpy as np
from ultralytics import YOLO

class HelmetDetector:
    def __init__(self, model_path, video_path, output_path, confidence_threshold=0.7):  # Increased default confidence
        # Initialize YOLO model
        self.model = YOLO(model_path)
        
        # Store confidence threshold
        self.confidence_threshold = confidence_threshold
        
        # Initialize video capture
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError("Error opening video file")
            
        # Get video properties
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer
        self.out = cv2.VideoWriter(output_path, 
                                 cv2.VideoWriter_fourcc(*'mp4v'), 
                                 self.fps, 
                                 (self.frame_width, self.frame_height))
        
        # Colors for visualization
        self.colors = {
            'With_Helmet': (0, 255, 0),    # Green
            'Without_Helmet': (0, 0, 255)   # Red
        }
        
        # Counters for statistics
        self.stats = {
            'With_Helmet': 0,
            'Without_Helmet': 0
        }
        
        # Track total detections and high confidence detections
        self.total_detections = 0
        self.high_conf_detections = 0

    def draw_detection_box(self, frame, box, cls, conf):
        """Draw detection box with label and confidence"""
        x1, y1, x2, y2 = map(int, box)
        color = self.colors[cls]
        
        # Draw box with thicker line for higher confidence
        thickness = 3 if conf > 0.8 else 2  # Thicker line for very high confidence
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
        
        # Create label with class and confidence percentage
        label = f'{cls} {conf*100:.1f}%'
        
        # Get label size
        (label_width, label_height), _ = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        
        # Draw label background
        cv2.rectangle(frame, 
                     (x1, y1 - label_height - 10), 
                     (x1 + label_width + 10, y1), 
                     color, 
                     -1)
        
        # Draw label text
        cv2.putText(frame, label, 
                    (x1 + 5, y1 - 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                    (255, 255, 255), 
                    2)

    def draw_stats(self, frame):
        """Draw detection statistics on frame"""
        # Background for stats
        cv2.rectangle(frame, 
                     (10, 10), 
                     (350, 140), 
                     (0, 0, 0), 
                     -1)
        
        # Draw stats text
        y_pos = 40
        for cls, count in self.stats.items():
            color = self.colors[cls]
            text = f'{cls}: {count}'
            cv2.putText(frame, text, 
                       (20, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       0.8, color, 2)
            y_pos += 30
        
        # Add confidence threshold info
        conf_text = f'Confidence Threshold: {self.confidence_threshold*100:.0f}%'
        cv2.putText(frame, conf_text,
                    (20, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (255, 255, 255), 2)

    def process_video(self):
        """Process the video and detect helmets"""
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
                
            # Reset counters for each frame
            self.stats = {
                'With_Helmet': 0,
                'Without_Helmet': 0
            }
            
            # Run detection with high confidence threshold
            results = self.model(frame, conf=self.confidence_threshold)
            
            # Process detections
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0]
                    
                    # Get class and confidence
                    cls = self.model.names[int(box.cls[0])]
                    conf = float(box.conf[0])
                    
                    # Only process high confidence detections
                    if conf >= self.confidence_threshold:
                        # Update statistics
                        self.stats[cls] += 1
                        
                        # Draw detection box
                        self.draw_detection_box(frame, [x1, y1, x2, y2], cls, conf)
            
            # Draw statistics
            self.draw_stats(frame)
            
            # Show frame
            cv2.imshow('Helmet Detection', frame)
            
            # Write frame to output video
            self.out.write(frame)
            
            # Break loop on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release resources
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()

def main():
    # Paths configuration
    model_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/best1.pt'
    video_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/tripletest.mp4'
    output_path = 'C:/Users/DELL/Desktop/only traffic project/helmet detection/testOUTPUT3.mp4'
    
    # Create detector with high confidence threshold (0.7 = 70%)
    # You can adjust this value between 0 and 1
    # Higher values (like 0.8 or 0.9) will give fewer but more confident detections
    detector = HelmetDetector(
        model_path=model_path,
        video_path=video_path,
        output_path=output_path,
        confidence_threshold=0.7  # Adjust this value as needed
    )
    
    detector.process_video()

if __name__ == "__main__":
    main()


0: 384x640 3 With_Helmets, 18.3ms
Speed: 1.6ms preprocess, 18.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 With_Helmets, 21.5ms
Speed: 0.0ms preprocess, 21.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 With_Helmets, 16.3ms
Speed: 1.7ms preprocess, 16.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With_Helmets, 16.4ms
Speed: 1.2ms preprocess, 16.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With_Helmets, 16.5ms
Speed: 1.0ms preprocess, 16.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With_Helmets, 16.3ms
Speed: 1.0ms preprocess, 16.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With_Helmets, 16.1ms
Speed: 1.0ms preprocess, 16.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With_Helmets, 18.5ms
Speed: 0.0ms preprocess, 18.5ms inference